In [ ]:
# -*- coding: utf-8 -*-
"""
🚀 Vintern Embedding API Server - BẢN BYPASS (Final Fix)
⚠️ YÊU CẦU: Đã Restart Runtime trước khi chạy!
"""

import sys
import os
import shutil
from unittest.mock import MagicMock

# ============================================================================
# BƯỚC 0: TẠO "THUỐC GIẢ" (MOCKING FLASH_ATTN) - QUAN TRỌNG NHẤT
# ============================================================================
print("💊 Đang tiêm 'thuốc giả' Flash Attention để đánh lừa bộ kiểm tra...")

# 1. Tạo một module giả
mock_flash_attn = MagicMock()
# Fix lỗi "__spec__ is not set" bằng cách gán spec giả
mock_flash_attn.__spec__ = MagicMock()

# 2. Gán vào hệ thống (Python sẽ nghĩ là thư viện này đã được cài)
sys.modules["flash_attn"] = mock_flash_attn
sys.modules["flash_attn.flash_attn_interface"] = mock_flash_attn
sys.modules["flash_attn.bert_padding"] = mock_flash_attn

print("✅ Đã bypass thành công bước kiểm tra thư viện!")

# ============================================================================
# BƯỚC 1: DỌN DẸP CACHE (ĐỂ TRÁNH XUNG ĐỘT CŨ)
# ============================================================================
cache_dir = "/root/.cache/huggingface/modules/transformers_modules/5CD-AI/Vintern-Embedding-1B"
if os.path.exists(cache_dir):
    print("🗑️ Đang xóa cache model cũ...")
    shutil.rmtree(cache_dir, ignore_errors=True)

# ============================================================================
# BƯỚC 2: CÀI ĐẶT DEPENDENCIES (NHANH GỌN)
# ============================================================================
print("📦 Đang cài đặt packages cần thiết...")
import subprocess

packages = [
    "transformers==4.48.0",
    "torch",
    "torchvision",
    "Pillow",
    "flask",
    "flask-cors",
    "pyngrok",
    "timm",
    "einops",
    "decord"
]

for package in packages:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])

print("✅ Packages đã sẵn sàng!")

# ============================================================================
# BƯỚC 3: LOAD MODEL (CHẾ ĐỘ EAGER)
# ============================================================================
import torch
from PIL import Image
from transformers import AutoModel, AutoProcessor
import numpy as np
import base64
import io
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok

model_name = "5CD-AI/Vintern-Embedding-1B"
device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"\n{'='*80}")
print(f"🔥 Device: {device.upper()}")
print(f"📥 Đang load model {model_name}...")
print(f"🛡️ Chế độ: Eager Attention (Bypass Flash Attn)")
print(f"{'='*80}\n")

processor = AutoProcessor.from_pretrained(model_name, trust_remote_code=True)

# Vì đã Mock ở Bước 0, transformers sẽ cho qua bước check import
# attn_implementation="eager" sẽ đảm bảo model chạy logic thường, không gọi vào cái Mock
model = AutoModel.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16 if device == "cuda" else torch.float32,
    low_cpu_mem_usage=True,
    trust_remote_code=True,
    attn_implementation="eager"
).eval()

if device == "cuda":
    model = model.cuda()

print(f"\n✅ Model loaded successfully! (Server đã lên)\n")

# ============================================================================
# BƯỚC 4: API SERVER
# ============================================================================
def base64_to_image(base64_str):
    img_bytes = base64.b64decode(base64_str)
    return Image.open(io.BytesIO(img_bytes))

def tensor_to_base64(tensor):
    buffer = io.BytesIO()
    numpy_array = tensor.cpu().float().numpy()
    np.save(buffer, numpy_array)
    return base64.b64encode(buffer.getvalue()).decode('utf-8')

def base64_to_tensor(base64_str):
    tensor_bytes = base64.b64decode(base64_str)
    buffer = io.BytesIO(tensor_bytes)
    numpy_array = np.load(buffer, allow_pickle=False)
    tensor = torch.from_numpy(numpy_array)
    if device == "cuda":
        tensor = tensor.cuda()
    return tensor

app = Flask(__name__)
CORS(app)

@app.route('/health', methods=['GET'])
def health():
    return jsonify({"status": "ok", "device": device})

@app.route('/encode_images', methods=['POST'])
def encode_images():
    try:
        data = request.json
        images_b64 = data['images']
        images = [base64_to_image(img_b64) for img_b64 in images_b64]
        batch_images = processor.process_images(images)
        if device == "cuda":
            batch_images["pixel_values"] = batch_images["pixel_values"].cuda().bfloat16()
            batch_images["input_ids"] = batch_images["input_ids"].cuda()
            batch_images["attention_mask"] = batch_images["attention_mask"].cuda().bfloat16()
        else:
            batch_images["pixel_values"] = batch_images["pixel_values"].float()
            batch_images["attention_mask"] = batch_images["attention_mask"].float()
        with torch.no_grad():
            embeddings = model(**batch_images)
        return jsonify({"embeddings": [tensor_to_base64(embeddings[i]) for i in range(len(images))]})
    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route('/encode_texts', methods=['POST'])
def encode_texts():
    try:
        data = request.json
        texts = data['texts']
        batch_texts = processor.process_docs(texts)
        if device == "cuda":
            batch_texts["input_ids"] = batch_texts["input_ids"].cuda()
            batch_texts["attention_mask"] = batch_texts["attention_mask"].cuda().bfloat16()
        else:
            batch_texts["attention_mask"] = batch_texts["attention_mask"].float()
        with torch.no_grad():
            embeddings = model(**batch_texts)
        return jsonify({"embeddings": [tensor_to_base64(embeddings[i]) for i in range(len(texts))]})
    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route('/encode_query', methods=['POST'])
def encode_query():
    try:
        data = request.json
        query = data['query']
        batch_query = processor.process_queries([query])
        if device == "cuda":
            batch_query["input_ids"] = batch_query["input_ids"].cuda()
            batch_query["attention_mask"] = batch_query["attention_mask"].cuda().bfloat16()
        else:
            batch_query["attention_mask"] = batch_query["attention_mask"].float()
        with torch.no_grad():
            embedding = model(**batch_query)
        return jsonify({"embedding": tensor_to_base64(embedding)})
    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route('/compute_similarity', methods=['POST'])
def compute_similarity():
    try:
        data = request.json
        query_emb_b64 = data['query_embedding']
        docs_emb_b64 = data['doc_embeddings']
        query_embedding = base64_to_tensor(query_emb_b64)
        doc_embeddings = [base64_to_tensor(emb_b64) for emb_b64 in docs_emb_b64]
        scores = processor.score_multi_vector(query_embedding, doc_embeddings)
        return jsonify({"scores": tensor_to_base64(scores[0])})
    except Exception as e:
        return jsonify({"error": str(e)}), 500

# ============================================================================
# BƯỚC 5: NGROK
# ============================================================================
print("\n" + "="*80)
print("🌐 Đang khởi động Ngrok...")
NGROK_AUTH_TOKEN = "thay_đổi_bằng_token_của_bạn_nhé"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
ngrok.kill()
public_url = ngrok.connect(5000)

print(f"🎉 SERVER ĐÃ CHẠY THÀNH CÔNG!")
print(f"🌐 Public URL: {public_url}")
print("="*80 + "\n")

app.run(port=5000)